In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils import data
from torch.autograd import Variable

In [237]:
dataset = pd.read_csv(r'net2input.csv')
dataset_training = dataset.truncate(after=99).reset_index(drop=True).to_numpy()
dataset_testing = dataset.truncate(before=101, after=150).reset_index(drop=True).to_numpy()
training_tensor_dataset = data.TensorDataset(tensor_1)
testing_tensor_dataset = data.TensorDataset(tensor_2)

train_loader = torch.utils.data.DataLoader(dataset=dataset_training, batch_size = 5, shuffle = True)
testing_loader = torch.utils.data.DataLoader(dataset=dataset_testing, batch_size = 5, shuffle = True)
print(len(train_loader))
print(len(testing_loader))

input_size = 9
output_size = dataset["move_class"].nunique()
print(output_size)
hidden_size = 10


20
10
15


In [238]:
dataset

,index,start_time,duration,section,tempo,danceability,energy,valence,move_class
0,0,8.64248,0.53227,0,113.255,0.935,0.526,0.854,0
1,1,9.17475,0.53417,0,113.255,0.935,0.526,0.854,0
2,2,9.70893,0.53276,0,113.255,0.935,0.526,0.854,0
3,3,10.24169,0.53135,0,113.255,0.935,0.526,0.854,0
4,4,10.77303,0.52898,0,113.255,0.935,0.526,0.854,0
...,...,...,...,...,...,...,...,...,...
60894,485,206.29409,0.41231,7,144.041,0.687,0.514,0.494,3
60895,486,206.70641,0.41176,7,144.041,0.687,0.514,0.494,3
60896,487,207.11817,0.41164,7,144.041,0.687,0.514,0.494,3
60897,488,207.52981,0.41433,7,144.041,0.687,0.514,0.494,3


In [239]:
class Neural_Network(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize):
        super(Neural_Network, self).__init__()
        self.fc1 = nn.Linear(inputSize, hiddenSize)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hiddenSize, outputSize)
        
    def forward(self, X):
        out = self.fc1(X)
        out = self.relu(out)
        out = self.fc2(out)
        return out

    #def backward(self, X, y, o):
     #   self.o_error = y - o # error in output
      #  self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
       # self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        #self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        #self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        #self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
    
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)

In [240]:
NN = Neural_Network(input_size, hidden_size, output_size)

# loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
learning_rate = 0.001
optimizer = torch.optim.SGD(NN.parameters(), lr=learning_rate, nesterov=True, momentum=0.9, dampening=0)

In [241]:
num_epochs = 500

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

NN = NN.float()
for epoch in range(num_epochs):
    
    train_correct = 0
    train_total = 0
    
    for i, batch in enumerate(train_loader):
        
        items = Variable(batch[0])
        classes = Variable(batch[1])
        
        NN.train()
        
        optimizer.zero_grad()
        outputs = NN(items.float())
        print(outputs)
        print(classes)
        loss = criterion(outputs, classes)
        optimizer.step()
        
        train_total += classes.size(0)
        #_, predicted = torch.max(outputs.data, 1)
        #train_correct += (predicted == classes.data).sum()
        
        print("Epoch %d/%d, Iteration %d/%d"
                 %(epoch+1, num_epochs, i+1, len(dataset_training)//5))
        print(loss)

        #NN.eval()
        
        #train_loss.append(loss.data[0])
        #train_accuracy.append((100 * train_correct / train_total))
        
        #test_items = torch.FloatTensor()


tensor([ -0.7324,  14.0341,  20.0300,   3.4897,  -0.6845, -18.6073, -19.1196,
         -6.2696,  -7.8233,   5.6816,  -9.2599,  -0.8805,  -3.0958,  -4.8302,
        -10.0256], grad_fn=<AddBackward0>)
tensor([ 40.0000,  29.8845,   0.5304,   1.0000, 112.9980,   0.9350,   0.5260,
          0.8540,   8.0000], dtype=torch.float64)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)